#### Payroll Attendacne - Amit (Original)

In [186]:
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe
import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np

In [187]:
import warnings
warnings.filterwarnings("ignore", message=".*SQLAlchemy connectable.*")

In [188]:
# Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:\Users\Administrator\LIS\Kenvue\python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)

In [189]:
# Python Master for ISPStore
sheet = client.open_by_key("1T_fuAUdJw-DB9dn9v3ZRg5YZ3mruCtg3Ghkd1mhVBRM") # Master Sheets ( Dimensions Sheet)

In [190]:
Schema_Master = sheet.worksheet("Schema_Master")
Schema_Master = get_as_dataframe(Schema_Master)
Data = sheet.worksheet("Data")

In [191]:
Schema_Master= Schema_Master[Schema_Master['flag']==1]
Schema_Master.head()

Project            SQL Schema Name From Date    To Date  flag  flag2
0    Amul GT & MT           [AmulManagement]  1-Dec-25  31-Dec-25   1.0    1.0
1     AWL Agri-GT    [AdaniWilmerManagement]  1-Dec-25  31-Dec-25   1.0    1.0
2  Bacardi Legacy  [BacardilegacyManagement]  1-Dec-25  31-Dec-25   1.0    1.0
3        Bajaj MT        [BajajMTManagement]  1-Dec-25  31-Dec-25   1.0    1.0
4  Bajaj Consumer  [BajajConsumerManagement]  1-Dec-25  31-Dec-25   1.0    1.0

In [192]:
x= Schema_Master['SQL Schema Name'].to_list()
projects = [str(p).strip("[]") for p in Schema_Master['SQL Schema Name'].tolist()]


In [193]:
projects

['AmulManagement',
 'AdaniWilmerManagement',
 'BacardilegacyManagement',
 'BajajMTManagement',
 'BajajConsumerManagement',
 'IDAMManagement',
 'BrillonMTManagement',
 'CamlinManagement',
 'CholayilManagement',
 'DaburPharmaManagement',
 'DanoneGTManagement',
 'DanoneMTManagement',
 'DelMonteManagement',
 'EvereadyManagement',
 'GCPLManagement',
 'GCPL_SAMTManagement',
 'GCPLFoodsManagement',
 'HaleonManagement',
 'HimalayaTurmericGTManagement',
 'HimalayaTurmericManagement',
 'Jayanti_GroupManagement',
 'JohnsonJohnsonManagement',
 'KRBLMTManagement',
 'MarsManagement',
 'McnroeManagement',
 'MTRManagement',
 'ParleManagement',
 'PidiliteManagement',
 'PiramalManagement',
 'ReckittManagement',
 'RelianceManagement',
 'BeamSuntoryManagement',
 'WiproGranammaManagement',
 'ZydusMTManagement',
 'ZydusGTManagement',
 'AbbottManagement']

In [194]:
date_ranges = [
    (
        pd.to_datetime(row["From Date"]).strftime("%Y-%m-%d"),
        pd.to_datetime(row["To Date"]).strftime("%Y-%m-%d")
    )
    for _, row in Schema_Master.iterrows()
]


In [195]:
server = "157.119.230.120,4071"
database = "PowerBI_LISV2"
username = "sa"
password = "Y0m@Sql.!.123"

conn_str = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
UID={username};
PWD={password};
TrustServerCertificate=yes;
"""

In [196]:
conn = pyodbc.connect(conn_str)

In [197]:
# x = ['[CholayilManagement]','[ParleManagement]']

In [198]:
final_df = pd.DataFrame()

In [199]:
# date_ranges = [
#     ("2025-07-21", "2025-07-22"),
#      ("2025-08-22","2025-08-24")
# ]

In [200]:
pivot_frames = []

In [201]:
for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
        query = f""" 
WITH CTE AS (
    SELECT
        '{project}' AS SchemaName,
        A.UserName,
        M.EmployeeID,
        M.FullName,
        TRY_CONVERT(date, M.DateofJoining, 105) AS DateofJoining,
        M.Region,
        M.State,
        M.City,
        M.Designation,
        TRY_CONVERT(date, M.LastWorkingDate, 105) AS LastWorkingDate,
        M.Status AS EmpStatus,
        -- A.Status AS AttRawStatus,
        -- A.ApprovalStatus,
        CASE
            WHEN A.Status = 'Present' THEN 'P'
            WHEN A.Status = 'Absent' OR A.Status IS NULL THEN 'A'
            WHEN A.Status = 'Leave' THEN 'L'
            WHEN A.Status IN ('Compensatory Off','Comp Off') THEN 'CO'
            WHEN A.Status IN ('Week Off','WeekOff') THEN 'WO'
            WHEN A.Status IN ('Meeting','Meeting/Training') THEN 'M'
            WHEN A.Status = 'Training' THEN 'T'
            WHEN A.Status IN ('National Holiday','National Hoilday','Holiday','Regional Holiday') THEN 'H'
            WHEN A.Status IN ('Bandh/Strike','Bandh Strike','Bandh / Strike','Market closed','Strike') THEN 'B'
            WHEN A.Status = 'Vacant' THEN 'V'
            WHEN A.Status IN ('Personal','Personal Leave') THEN 'PL'
            WHEN A.Status = 'Casual Leave' THEN 'CL'
            WHEN A.Status = 'Half Day' THEN 'HD'
            ELSE 'Condition not found'
        END AS AttStatus,
        DATEDIFF(
            DAY, 
            CAST('{date_from}' AS DATE), 
            TRY_CONVERT(date, A.[Date], 105)
        ) + 1 AS DayNum, 
        -- TRY_CONVERT(date, A.[Date], 105) AS Date2,
        RANK() OVER (PARTITION BY A.UserName,A.[Date] ORDER BY A.ISPAttendanceMasterExportId DESC) AS rk
    FROM PowerBI_LISV2.{value}.ISPAttendanceMasterExport AS A
    LEFT JOIN (SELECT K.* FROM
        (SELECT  *, RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK FROM 
        PowerBI_LISV2.{value}.UserMaster
        WHERE
        UserRole NOT IN ('MIS','RSM','SO','Program Manager','Assistant Manager','FO','Office Staff','ASM',
        'Senior Manager','Supervisor','National','Program Manager','ABI')
        OR Designation NOT in ('MIS','RSM','SO','Program Manager','Assistant Manager','FO','Office Staff','ASM',
        'Senior Manager','Supervisor','National','Program Manager','ABI')
        AND LOWER(UserName) NOT LIKE '%test%'
        AND LOWER(UserName) NOT LIKE '%sup%'
        AND LOWER(FullName) NOT LIKE '%test%'
        
        ) AS K
        WHERE K.RNK =1) AS M
        ON A.UserName = M.UserName
    WHERE 
        TRY_CONVERT(date, A.[Date], 105) BETWEEN '{date_from}' AND '{date_to}'
        AND M.UserRole NOT IN ('MIS','RSM','SO','Program Manager','Assistant Manager','FO','Office Staff','ASM',
        'Senior Manager','Supervisor','National','Program Manager','ABI')
        OR M.Designation NOT in ('MIS','RSM','SO','Program Manager','Assistant Manager','FO','Office Staff','ASM',
        'Senior Manager','Supervisor','National','Program Manager','ABI')
        AND LOWER(A.UserName) NOT LIKE '%test%'
        AND LOWER(A.UserName) NOT LIKE '%sup%'
        AND LOWER(A.FullName) NOT LIKE '%test%'
),
Filtered AS (
    SELECT *
    FROM CTE
    WHERE rk = 1
),
Pivoted AS (
    SELECT
        SchemaName,
        UserName,
        EmployeeID,
        FullName,
        DateofJoining,
        Region,
        State,
        City,
        Designation,
        LastWorkingDate,
        EmpStatus,
        -- day columns produced by the pivot
        [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
        [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
        [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
    FROM Filtered
    PIVOT (
        MAX(AttStatus) FOR DayNum IN (
            [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
            [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
            [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
        )
    ) AS P
)
SELECT *
FROM Pivoted;"""
        df = pd.read_sql(query, conn)
        final_df = pd.concat([final_df, df], ignore_index=True)
        print(f"Running for: {value} {date_from} {date_to}")
           

Running for: [AmulManagement] 2025-12-01 2025-12-31
Running for: [AdaniWilmerManagement] 2025-12-01 2025-12-31
Running for: [BacardilegacyManagement] 2025-12-01 2025-12-31
Running for: [BajajMTManagement] 2025-12-01 2025-12-31
Running for: [BajajConsumerManagement] 2025-12-01 2025-12-31
Running for: [IDAMManagement] 2025-12-01 2025-12-31
Running for: [BrillonMTManagement] 2025-12-01 2025-12-31
Running for: [CamlinManagement] 2025-12-01 2025-12-31
Running for: [CholayilManagement] 2025-12-01 2025-12-31
Running for: [DaburPharmaManagement] 2025-12-01 2025-12-31
Running for: [DanoneGTManagement] 2025-12-01 2025-12-31
Running for: [DanoneMTManagement] 2025-12-01 2025-12-31
Running for: [DelMonteManagement] 2025-12-01 2025-12-31
Running for: [EvereadyManagement] 2025-12-01 2025-12-31
Running for: [GCPLManagement] 2025-12-01 2025-12-31
Running for: [GCPL_SAMTManagement] 2025-12-01 2025-12-31
Running for: [GCPLFoodsManagement] 2025-12-01 2025-12-31
Running for: [HaleonManagement] 2025-12-01 2

In [202]:
# final_df2=final_df.drop_duplicates(['UserName','Date'])
final_df.head()

SchemaName  UserName    EmployeeID                  FullName  \
0  AmulManagement  AMLMR006          None                     Veena   
1  AmulManagement  AMLMR015  YM2023000616         Vijayalakshmi K R   
2  AmulManagement  AMLMR017          None                  Shobha R   
3  AmulManagement  AMLMR027  YM2023000754  Tejaswini Sachin Shedage   
4  AmulManagement  AMLMR028  YM2023000526                 Aruna H R   

  DateofJoining Region        State       City   Designation LastWorkingDate  \
0    2024-01-01  South    Karnataka  Bangalore  Merchandiser      2024-12-01   
1    2023-01-11  South    Karnataka  Bangalore  Merchandiser      2025-08-31   
2    2024-01-01  South    Karnataka  Bangalore  Merchandiser      2024-12-01   
3    2023-01-13   West  Maharashtra     Mumbai  Merchandiser            None   
4    2023-01-11  South    Karnataka  Bangalore  Merchandiser      2025-10-24   

   ...    22    23    24    25    26    27    28    29    30    31  
0  ...  None  None  None  None  None  None  None  None  None  None  
1  ...  None  None  None  None  None  None  None  None  None  None  
2  ...  None  None  None  None  None  None  None  None  None  None  
3  ...     P     P     P    WO  None  None  None  None  None  None  
4  ...  None  None  None  None  None  None  None  None  None  None  

[5 rows x 42 columns]

In [203]:
final_df2 = pd.merge(final_df,Schema_Master[['Project','From Date','To Date']], left_on = 'SchemaName' , right_on='Project', how = 'left')

In [204]:
final_df2.info() #256725
# 232885 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19144 entries, 0 to 19143
Data columns (total 45 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SchemaName       19144 non-null  object
 1   UserName         19144 non-null  object
 2   EmployeeID       15444 non-null  object
 3   FullName         19144 non-null  object
 4   DateofJoining    19135 non-null  object
 5   Region           19143 non-null  object
 6   State            19144 non-null  object
 7   City             19141 non-null  object
 8   Designation      19140 non-null  object
 9   LastWorkingDate  10270 non-null  object
 10  EmpStatus        19144 non-null  object
 11  1                8813 non-null   object
 12  2                8836 non-null   object
 13  3                8850 non-null   object
 14  4                8884 non-null   object
 15  5                9048 non-null   object
 16  6                9110 non-null   object
 17  7                9110 non-null 

In [205]:
final_df_uservisit = pd.DataFrame()

In [206]:
for value,project,(date_from,date_to) in zip(x,projects,date_ranges):
    uservisitquery = f"""
WITH CTE AS (
    SELECT
        '{project}' AS SchemaName,
        (TXT.UserName + '-' + TXT.Visit_Date) AS UserVisitKey,
        TXT.Visit_Date,
        TXT.UserName,
        (TXT.Visit_Date + ' ' + TXT.CHECKIN_TIME)  AS CheckinTime,
        TXT.LISStoreCode,
        (TXT.Visit_Date + '-' + TXT.LISStoreCode + '-' + TXT.UserName) AS StoreDateKey,
        TXT.StoreName,
        TXT.Designation,
        (TXT.Visit_Date + ' ' + TXT.CHECKOUT_TIME) AS CheckoutTime,
        TXT.FullName,
        TXT.NoPermissionReason,
        1 AS VisitFlag,
        DATEDIFF(
            DAY, 
            CAST('{date_from}' AS DATE), 
            TRY_CONVERT(date, TXT.Visit_Date, 105)
        ) + 1 AS DayNum
        -- DATEPART(DAY, TRY_CONVERT(date, TXT.Visit_Date, 105)) AS DayNum
    FROM (
        SELECT
            FORMAT(CONVERT(datetime, SA.[VisitDate], 105), 'dd-MM-yyyy') AS Visit_Date,
            SA.LISStoreCode,
            SA.StoreName,
            SA.UserName,
            SA.Designation,
            SA.FullName,
            CONVERT(varchar(8), CAST([CheckIn-Time]  AS time), 108) AS CHECKIN_TIME,
            CONVERT(varchar(8), CAST([CheckOut-Time] AS time), 108) AS CHECKOUT_TIME,
            SA.NoPermissionReason
        FROM [PowerBI_LISV2].{value}.[ISPStoreAttendance_NormalExport] AS SA
        LEFT JOIN (
            SELECT K.*
            FROM (
                SELECT *,
                       RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                FROM PowerBI_LISV2.{value}.UserMaster
            ) AS K
            WHERE K.RNK = 1
        ) AS UM
            ON UM.UserName = SA.UserName
        WHERE 
            TRY_CONVERT(date, SA.[VisitDate], 105) BETWEEN '{date_from}' AND '{date_to}'
            -- [Month] = MONTH(EOMONTH(GETDATE()))
            -- AND [Year]  = YEAR(EOMONTH(GETDATE()))
			-- True is considered here for all active users 
            -- AND UM.Status = 'True'
            AND SA.UserName NOT LIKE '%Star%'
            AND LOWER(SA.UserName) NOT LIKE '%test%'
    ) AS TXT
),
-- FILTERED is a table created to group by the user visit count and created a unique visit flag
Filtered AS (
    SELECT  
        -- CTE.UserVisitKey,
		CTE.UserName,
        CTE.DayNum,
        CASE WHEN COUNT(*) >= 1 THEN 1 ELSE 0 END AS VisitFlag
    FROM CTE
    GROUP BY 
	-- CTE.UserVisitKey,
	CTE.UserName, CTE.DayNum
),
-- creaing a pivot for day numbers at unique user level
Pivoted AS (
    SELECT
        UserName,
        [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
        [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
        [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
    FROM Filtered
    PIVOT (
        MAX(VisitFlag) FOR DayNum IN (
            [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
            [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
            [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31]
        )
    ) AS P
)
-- selecting from pivoted table and creating mtd total columns
SELECT 
    UserName,
    [1],[2],[3],[4],[5],[6],[7],[8],[9],[10],
    [11],[12],[13],[14],[15],[16],[17],[18],[19],[20],
    [21],[22],[23],[24],[25],[26],[27],[28],[29],[30],[31],
    -- Month-to-date total visits for the filtered month
    ISNULL([1],0)+ISNULL([2],0)+ISNULL([3],0)+ISNULL([4],0)+ISNULL([5],0)+
    ISNULL([6],0)+ISNULL([7],0)+ISNULL([8],0)+ISNULL([9],0)+ISNULL([10],0)+
    ISNULL([11],0)+ISNULL([12],0)+ISNULL([13],0)+ISNULL([14],0)+ISNULL([15],0)+
    ISNULL([16],0)+ISNULL([17],0)+ISNULL([18],0)+ISNULL([19],0)+ISNULL([20],0)+
    ISNULL([21],0)+ISNULL([22],0)+ISNULL([23],0)+ISNULL([24],0)+ISNULL([25],0)+
    ISNULL([26],0)+ISNULL([27],0)+ISNULL([28],0)+ISNULL([29],0)+ISNULL([30],0)+
    ISNULL([31],0) AS MTD_Total_Coverage
FROM Pivoted
ORDER BY UserName;"""
    df = pd.read_sql(uservisitquery, conn)
    final_df_uservisit = pd.concat([final_df_uservisit, df], ignore_index=True)
    print(f"Running for: {value} {date_from} {date_to}")


Running for: [AmulManagement] 2025-12-01 2025-12-31
Running for: [AdaniWilmerManagement] 2025-12-01 2025-12-31
Running for: [BacardilegacyManagement] 2025-12-01 2025-12-31
Running for: [BajajMTManagement] 2025-12-01 2025-12-31


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_26904\2729819963.py:101: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df_uservisit = pd.concat([final_df_uservisit, df], ignore_index=True)


Running for: [BajajConsumerManagement] 2025-12-01 2025-12-31
Running for: [IDAMManagement] 2025-12-01 2025-12-31
Running for: [BrillonMTManagement] 2025-12-01 2025-12-31
Running for: [CamlinManagement] 2025-12-01 2025-12-31
Running for: [CholayilManagement] 2025-12-01 2025-12-31
Running for: [DaburPharmaManagement] 2025-12-01 2025-12-31
Running for: [DanoneGTManagement] 2025-12-01 2025-12-31
Running for: [DanoneMTManagement] 2025-12-01 2025-12-31
Running for: [DelMonteManagement] 2025-12-01 2025-12-31
Running for: [EvereadyManagement] 2025-12-01 2025-12-31
Running for: [GCPLManagement] 2025-12-01 2025-12-31
Running for: [GCPL_SAMTManagement] 2025-12-01 2025-12-31
Running for: [GCPLFoodsManagement] 2025-12-01 2025-12-31
Running for: [HaleonManagement] 2025-12-01 2025-12-31
Running for: [HimalayaTurmericGTManagement] 2025-12-01 2025-12-31
Running for: [HimalayaTurmericManagement] 2025-12-01 2025-12-31
Running for: [Jayanti_GroupManagement] 2025-12-01 2025-12-31
Running for: [JohnsonJohns

In [207]:
# final_df.to_csv(r'C:\Users\Administrator\LIS\MISAutomation\Kunal\payrollattendance.csv')

In [208]:
final_df_uservisit=final_df_uservisit.fillna(0)

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_26904\384735082.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df_uservisit=final_df_uservisit.fillna(0)


In [209]:
final_df_uservisit.head()

UserName    1    2    3    4    5    6    7    8    9  ...   23   24   25  \
0  AMLMR027  1.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  ...  1.0  1.0  0.0   
1  AMLMR029  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  ...  0.0  1.0  1.0   
2  AMLMR040  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  ...  1.0  1.0  0.0   
3  AMLMR055  1.0  1.0  1.0  0.0  1.0  0.0  1.0  1.0  1.0  ...  1.0  1.0  0.0   
4  AMLMR063  1.0  0.0  1.0  1.0  0.0  1.0  1.0  1.0  0.0  ...  0.0  1.0  1.0   

   26  27  28  29  30  31  MTD_Total_Coverage  
0   0   0   0   0   0   0                  20  
1   0   0   0   0   0   0                  21  
2   0   0   0   0   0   0                  12  
3   0   0   0   0   0   0                  20  
4   0   0   0   0   0   0                  18  

[5 rows x 33 columns]

In [210]:
final_df3=pd.merge(final_df2,final_df_uservisit[['UserName','MTD_Total_Coverage']], on = 'UserName', how='left')

In [211]:
df_User_final =pd.DataFrame()

for value in x:
        query = f"""
        SELECT K.ISP, UM.FullName AS Supervisor
        FROM
        (SELECT DISTINCT ISP,Supervisor FROM PowerBI_LISV2.{value}.UserHierarchyListReport)AS K
        LEFT JOIN
        (SELECT K.UserName,K.FullName
            FROM (SELECT UserName,FullName,RANK() OVER (PARTITION BY UserName ORDER BY UserMasterId DESC) AS RNK
                    FROM PowerBI_LISV2.{value}.UserMaster
        			WHERE UserName LIKE '%SUP%'
                ) AS K
        WHERE K.RNK = 1) AS UM ON UM.UserName = K.Supervisor;
        """
        df_User = pd.read_sql(query, conn)
        df_User_final = pd.concat([df_User_final, df_User], ignore_index=True)

In [212]:
df_User_final.head()

ISP  Supervisor
0  AMLMR015  Pavithra T
1  AMLMR028  Pavithra T
2  AMLMR040  Pavithra T
3  AMLMR041  Pavithra T
4  AMLMR057  Pavithra T

In [213]:
final_df4 = pd.merge(final_df3,df_User_final,how='left',left_on = 'UserName', right_on = 'ISP')

In [214]:
final_df4 = final_df4.drop(columns = ['ISP'])

In [215]:
# final_df.to_csv(r'C:\Users\Administrator\LIS\MISAutomation\Kunal\payrollattendancepivot_df.csv')

In [216]:
#Sending df on sheet
Data.batch_clear(["A:AU"])
set_with_dataframe(Data, final_df4)